In [40]:
!pip install aiomoex
!pip install tinkoff-investments

Открытые вопросы:
* Стоит ли описывать варианты получения данных и причину выбора?

In [41]:
from tinkoff.invest import Client, AsyncClient, CandleInterval, SecurityTradingStatus
from tinkoff.invest.services import InstrumentsService
from tinkoff.invest.utils import quotation_to_decimal, now
from tinkoff.invest.caching.instruments_cache.instruments_cache import InstrumentsCache

import pandas as pd
from dotenv import load_dotenv

import os
# import asyncio
import time
from datetime import datetime, timezone, timedelta
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

In [42]:
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [43]:
load_dotenv()

TOKEN = "t.eRZ0sm51xUBVmE4xcdx4cebt0iTp8oIbMUFhdf2Ot7GOG7v0BjTryycaTnwiuyJbwpiqpLnKK6WNMCSdHiPZlg"
# os.getenv("TOKEN")

Подробная информация по тикеру.

**Не работает с индексами вроде индекса МосБиржи**

In [44]:
def get_info(ticker, TOKEN = TOKEN):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        tickers = []
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                tickers.append(
                    {
                        "name": item.name,
                        "ticker": item.ticker,
                        "class_code": item.class_code,
                        "figi": item.figi,
                        "uid": item.uid,
                        "type": method,
                        "min_price_increment": quotation_to_decimal(
                            item.min_price_increment
                        ),
                        "scale": 9 - len(str(item.min_price_increment.nano)) + 1,
                        "lot": item.lot,
                        "trading_status": str(
                            SecurityTradingStatus(item.trading_status).name
                        ),
                        "api_trade_available_flag": item.api_trade_available_flag,
                        "currency": item.currency,
                        "exchange": item.exchange,
                        "buy_available_flag": item.buy_available_flag,
                        "sell_available_flag": item.sell_available_flag,
                        "short_enabled_flag": item.short_enabled_flag,
                        "klong": quotation_to_decimal(item.klong),
                        "kshort": quotation_to_decimal(item.kshort),
                    }
                )

        tickers_df = pd.DataFrame(tickers)

        ticker_df = tickers_df[tickers_df["ticker"] == ticker]
        if ticker_df.empty:
            print("There is no such ticker: %s", ticker)
            # return

        figi = ticker_df["figi"].iloc[0]
        print(f"\nTicker {ticker} have figi={figi}\n")
        print(f"Additional info for this {ticker} ticker:")
        print(ticker_df.iloc[0])

        return figi

ticker = "SPY"
get_info(ticker)


Ticker SPY have figi=BBG000BDTBL9

Additional info for this SPY ticker:
name                                                   SPDR S&P 500 ETF Trust
ticker                                                                    SPY
class_code                                                              SPBXM
figi                                                             BBG000BDTBL9
uid                                      59ba0c48-4f13-429a-b1a1-6aff71baca05
type                                                                     etfs
min_price_increment                                                0.01000000
scale                                                                       2
lot                                                                         1
trading_status              SECURITY_TRADING_STATUS_NOT_AVAILABLE_FOR_TRADING
api_trade_available_flag                                                 True
currency                                                             

'BBG000BDTBL9'

Получить только FIGI.

In [45]:
def get_figi(ticker, TOKEN = TOKEN):
    with Client(TOKEN) as client:
        instruments: InstrumentsService = client.instruments
        for method in ["shares", "bonds", "etfs", "currencies", "futures"]:
            for item in getattr(instruments, method)().instruments:
                return item.figi

ticker = "SPY"
get_figi(ticker)

'BBG000BBCQD7'

In [46]:
def get_5min(ticker: str) -> pd.DataFrame:
    ticker = ticker.upper()
    file_path = DATA_DIR / f"{ticker}_5min.parquet"

    # Если файл есть — читаем и сразу делаем DateTime колонкой
    if file_path.exists():
        print(f"Загружаю из кэша: {ticker}")
        df = pd.read_parquet(file_path)
        if df.index.name == "DateTime":
            df = df.reset_index()
        return df.sort_values("DateTime", ascending=False)

    # Если файла нет — скачиваем
    print(f"Скачиваю {ticker} с 01.01.2025...")
    figi = get_figi(ticker)
    new_rows = []
    current = datetime(2025, 1, 1)
    now = datetime.now()

    with Client(TOKEN) as client:
        while current < now:
            end = min(current + timedelta(days=7), now - timedelta(minutes=10))

            if end <= current:
                break

            for _ in range(3):
                try:
                    candles = client.market_data.get_candles(
                        figi=figi,
                        from_=current,
                        to=end,
                        interval=CandleInterval.CANDLE_INTERVAL_5_MIN
                    ).candles

                    for c in candles:
                        new_rows.append({
                            "DateTime": c.time,
                            "Open":  c.open.units  + c.open.nano  / 1e9,
                            "High":  c.high.units  + c.high.nano  / 1e9,
                            "Low":   c.low.units   + c.low.nano   / 1e9,
                            "Close": c.close.units + c.close.nano / 1e9,
                            "Volume": c.volume
                        })
                    print(f"  +{len(candles)} свечей → {end.strftime('%Y-%m-%d %H:%M')}")
                    break
                except Exception as e:
                    if "RESOURCE_EXHAUSTED" in str(e):
                        print("  Лимит! Жду 45 сек...")
                        time.sleep(45)
                    else:
                        time.sleep(5)
            else:
                print(f"  Не удалось скачать кусок {current.date()} → {end.date()}")
            
            current = end
            time.sleep(1.2)

    if not new_rows:
        raise RuntimeError(f"Не удалось скачать данные для {ticker}")

    df = pd.DataFrame(new_rows)
    df = df.sort_values("DateTime", ascending=False)
    
    df.to_parquet(file_path, compression="zstd", index=False)
    print(f"Сохранено: {file_path} → {len(df):,} строк")

    return df

In [52]:
df.head(10)

,DateTime,Open,High,Low,Close,Volume
0,2025-11-21 20:55:00+00:00,27.91,27.94,27.86,27.89,708742
1,2025-11-21 20:50:00+00:00,27.98,28.02,27.89,27.91,70367
2,2025-11-21 20:45:00+00:00,27.98,28.02,27.97,27.99,26310
3,2025-11-21 20:40:00+00:00,28.05,28.05,27.93,27.98,71253
4,2025-11-21 20:35:00+00:00,27.99,28.05,27.99,28.05,26110
5,2025-11-21 20:30:00+00:00,27.97,27.99,27.96,27.99,21946
6,2025-11-21 20:25:00+00:00,28.01,28.01,27.96,27.97,362723
7,2025-11-21 20:20:00+00:00,28.00,28.01,27.98,28.00,342377
8,2025-11-21 20:15:00+00:00,28.03,28.03,27.97,28.00,28372
9,2025-11-21 20:10:00+00:00,28.06,28.07,28.03,28.04,9760
